In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('../')

In [3]:
from models import resnet_v1, resnet_v2
from utils import (
    setup, data, viz
)
from utils.training import train
from utils.testing import test

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [4]:
setup.set_seed()
cuda = setup.is_cuda()
device = setup.get_device()

[INFO] seed set 1
[INFO] Cuda Avaliable :  False
[INFO] device :  cpu


In [5]:
train_loader = data.CIFAR10_dataset(
    train=True, cuda=cuda
).get_loader()
test_loader = data.CIFAR10_dataset(
    train=False, cuda=cuda
).get_loader()

Files already downloaded and verified
[INFO] train dataset of size 50000 loaded...
Files already downloaded and verified
[INFO] test dataset of size 10000 loaded...


In [12]:
# model = resnet_v1.ResNet18
model = resnet_v2.ResNet18
net = model().to(device)
viz.show_model_summary(net, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 17, 17]          73,728
      BatchNorm2d-14          [-1, 128,

In [7]:
## 11,173,962

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    net.parameters(), lr=0.01,
    momentum=0.9, weight_decay=5e-4
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

EPOCHS = 2

In [16]:
for epoch in range(EPOCHS):
    train_batch_loss, train_batch_acc= train(
        net, device, 
        train_loader, optimizer, criterion, epoch,
    )
    train_loss = np.mean(train_batch_loss)
    train_acc = np.mean(train_batch_acc)
    test_loss, test_acc = test(
        net, device,
        test_loader, criterion, epoch,
    )
    scheduler.step()

  0%|          | 0/391 [00:00<?, ?it/s]

TEST         Loss:0.0043         Acc:80.75         [8075 / 10000]


TRAIN Epoch:1 Loss:0.5996 Batch:390 Acc:79.01: 100%|██████████| 391/391 [00:28<00:00, 13.67it/s]


TEST         Loss:0.0041         Acc:81.65         [8165 / 10000]


In [17]:
train_loss, train_acc, test_loss, test_acc

(0.6048094331455962, 78.86345923509285, 0.004068706184625626, 81.65)